In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.types import *
# import pyspark.sql.functions as F
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

# Q1

In [2]:
spark = SparkSession.builder.config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").appName("ass3_q1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/02 03:55:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd1 = spark.sparkContext.textFile("data/departuredelays.csv") 
rdd1.take(5)

['date,delay,distance,origin,destination',
 '01011245,6,602,ABE,ATL',
 '01020600,-8,369,ABE,DTW',
 '01021245,-2,602,ABE,ATL',
 '01020605,-4,602,ABE,ATL']

In [4]:
rdd1 = rdd1.filter(lambda x: "delay" not in x)
rdd1.take(3)

['01011245,6,602,ABE,ATL',
 '01020600,-8,369,ABE,DTW',
 '01021245,-2,602,ABE,ATL']

We can see the column name of each column in the dataset and from left to right are `date, delay, distance, origin, destination`. According to the question, we need to find the rows whose `origin` is `ATL`, which will be partitioned into one partition and the rest will be partitioned into another partition.

In [5]:
rdd2 = rdd1.keyBy(lambda x: 1 if x.split(",")[3] == "ATL" else 0)
rdd2.take(3)

[(0, '01011245,6,602,ABE,ATL'),
 (0, '01020600,-8,369,ABE,DTW'),
 (0, '01021245,-2,602,ABE,ATL')]

In [6]:
import random
random.seed(42)

def partition_func(key):
    if key == 1:        
        return 0
    else:
        return random.randint(1, 3)
        # return int(hash(key) % 3) + 1

partitioned_rdd = rdd2.partitionBy(4, partition_func)

In [7]:
partitioned_rdd = partitioned_rdd.map(lambda x: x[1])
partition_list = partitioned_rdd.glom().collect()

In [8]:
print("The number of elements of 4 partitons in sequence: ",[len(x) for x in partition_list])
print("Some elements in partition 1: \n", partition_list[0][0:5])
print("Some elements in partition 2: \n", partition_list[1][0:5])
print("Some elements in partition 3: \n", partition_list[2][0:5])
print("Some elements in partition 4: \n", partition_list[3][0:5])

The number of elements of 4 partitons in sequence:  [91484, 434701, 432274, 433119]
Some elements in partition 1: 
 ['01010640,-4,517,ATL,MIA', '01011925,-1,636,ATL,DFW', '01011245,22,636,ATL,DFW', '01011405,-3,636,ATL,DFW', '01011540,-4,636,ATL,DFW']
Some elements in partition 2: 
 ['01020600,-8,369,ABE,DTW', '01021245,-2,602,ABE,ATL', '01020605,-4,602,ABE,ATL', '01041243,10,602,ABE,ATL', '01040605,28,602,ABE,ATL']
Some elements in partition 3: 
 ['01050605,9,602,ABE,ATL', '01061725,69,602,ABE,ATL', '01061230,0,369,ABE,DTW', '01071725,0,602,ABE,ATL', '01071219,0,569,ABE,ORD']
Some elements in partition 4: 
 ['01011245,6,602,ABE,ATL', '01031245,-4,602,ABE,ATL', '01030605,0,602,ABE,ATL', '01051245,88,602,ABE,ATL', '01061215,-6,602,ABE,ATL']
